# EDA: тексты запросов по сессиям 1.5gb

In [1]:
# тексты запросов, где только цифры, с цифрами, короткие, длинные запросы (отдельно по сессиям)
# - количество слов в запросе, бренды (отдельно по сессиям)
# - как часто полностью совпадает с названием категорий (по расс левенштейна/напрямую)

In [2]:
import json

with open("sessions_420.json") as js:
    searches = json.load(js)
searchesLength = len(searches)

In [3]:
queryLengths = {}
queryWordQuantities = {}

for i in range(searchesLength):
    lwords = len(searches[i]["query"].split())
    if not (lwords in queryWordQuantities):
        queryWordQuantities[lwords] = 1
    else:
        queryWordQuantities[lwords] += 1
    l = len(searches[i]["query"])
    if not (l in queryLengths):
        queryLengths[l] = 1
    else:
        queryLengths[l] += 1
        
queryWordQuantities = {k: v for k, v in sorted(queryWordQuantities.items(), key = lambda item: item[0])}
queryLengths = {k: v for k, v in sorted(queryLengths.items(), key = lambda item: item[0])}

In [4]:
from math import log

sum = 0
print("Количество символов в запросе:")
for k, v in queryLengths.items():
    sum += k * v
    print("{0:3} : {1:4} : ".format(k, v), "*" * int(log(v, 1.2)))
print("\nВ среднем: {0}".format(sum // searchesLength))

Количество символов в запросе:
  1 :  491 :  *********************************
  2 : 4078 :  *********************************************
  3 : 15172 :  ****************************************************
  4 : 63387 :  ************************************************************
  5 : 108606 :  ***************************************************************
  6 : 75076 :  *************************************************************
  7 : 90543 :  **************************************************************
  8 : 87857 :  **************************************************************
  9 : 92667 :  **************************************************************
 10 : 119050 :  ****************************************************************
 11 : 80912 :  *************************************************************
 12 : 108028 :  ***************************************************************
 13 : 165830 :  *****************************************************************
 14 : 

In [5]:
print("Количество слов в запросе:")
for k, v in queryWordQuantities.items():
    print("{0:3} : {1:5} : ".format(k, v), "*" * int(log(v, 1.2)))

Количество слов в запросе:
  1 : 521216 :  ************************************************************************
  2 : 696259 :  *************************************************************************
  3 : 647138 :  *************************************************************************
  4 : 309899 :  *********************************************************************
  5 : 119192 :  ****************************************************************
  6 : 58114 :  ************************************************************
  7 : 19798 :  ******************************************************
  8 : 13164 :  ****************************************************
  9 : 11252 :  ***************************************************
 10 :  6066 :  ***********************************************
 11 :  4877 :  **********************************************
 12 :  2638 :  *******************************************
 13 :  1910 :  *****************************************
 14 :  2002 :  

In [6]:
import re

brands = []
with open("brands.txt") as raw:
    for row in raw:
        brands.append(re.sub("[^а-яА-Яa-zA-Z0-9]", "", row).lower())
brands = sorted(list(set(brands)))

In [7]:
categories = []
with open("categories.txt") as raw:
    for row in raw:
        categories.append(re.sub("[^а-яА-Яa-zA-Z0-9]", "", row).lower())
categories = sorted(list(set(categories)))

In [8]:
#%pip install python-Levenshtein
from Levenshtein import distance as lev

In [9]:
errorful = []
numerical = 0
containingNumbers = []
containingBrands = []
onlyBrands = 0
equalToCategory = 0
similarToCategory = []
for i in range(searchesLength):
    q = searches[i]["query"]
    qStripped = re.sub("[^а-яА-Яa-zA-Z0-9]", "", q)
    if (not qStripped):
        errorful.append(q)
    elif (qStripped == re.sub("[^0-9]", "", qStripped)):
        numerical += 1
    else:
        if (qStripped != re.sub("[^а-яА-Яa-zA-Z]", "", qStripped)):
            containingNumbers.append(q)
        qLower = q.lower()
       
        containsBrand = False
        for brand in brands:
            if (qLower.find(brand) != -1):
                containsBrand = True
                if (qLower == brand):
                    onlyBrands += 1
                break
        if (containsBrand):
            containingBrands.append(q)
        
        mivLev = len(qLower)
        for category in categories:
            if (lev(qLower, category) < len(qLower) // 3):
                similarToCategory.append(q)
                if (qLower == category):
                    equalToCategory += 1
                    break

In [10]:
def percent(val):
    return val * 100 // searchesLength

print("Запросов, содержащих цифры -- {0} ({1}%). Из них состоящих из цифр -- {2} ({3}%)."
      .format(len(containingNumbers), percent(len(containingNumbers)), 
              numerical, percent(numerical)))
print("Запросов, содержащих названия бренда -- {0} ({1}%). Из них состоящих только из названия -- {2} ({3}%)."
      .format(len(containingBrands), percent(len(containingBrands)),
             onlyBrands, percent(onlyBrands)))
print("Запросов, совпадающих с названием категории -- {0} ({1}%)."
      .format(equalToCategory, percent(equalToCategory)))
print("Запросов, похожих на название категории (с малым расстоянием Левентшейна) -- {0} ({1}%)."
      .format(len(similarToCategory), percent(len(similarToCategory))))

Запросов, содержащих цифры -- 94140 (3%). Из них состоящих из цифр -- 18828 (0%).
Запросов, содержащих названия бренда -- 516084 (21%). Из них состоящих только из названия -- 1922 (0%).
Запросов, совпадающих с названием категории -- 123572 (5%).
Запросов, похожих на название категории (с малым расстоянием Левентшейна) -- 453708 (18%).


In [11]:
print(errorful)

['娇兰', '粘贴布朗维', '阿玛尼', '阿玛尼', '精华油', '娇兰', '资生堂', '娇兰', '定妆喷雾', '伊丽莎白', '布鲁克', '头发拉直器', '卸妆膏', '卸妆', '雅诗兰黛', '夏洛特·蒂伯里', '拉蒂森香料厂', 'برجوا اير مات فاونديشن', '娇韵诗', '黑绷带', '嗯', '科颜氏', 'նըղ', '保湿水', '双萃', '𝕓𝕒𝕥𝕚𝕤𝕥𝕖', '眼霜', '娇韵诗', '娇兰', '大自然的伊夫·罗彻', '雅诗兰黛', '眼霜', '𝖫𝖺𝗆𝖾𝗅 𝗒𝗈𝗎𝗋 𝗌𝖾𝖼𝗋𝖾𝗍', '眼霜', '永远化妆', '欧莱雅', '巴斯吉护理', '琥珀蜜蜡', '哑光唇膏', 'ĺ', '帕尔马', 'ㄹ겨얌', '阿玛尼', '娇兰', 'նըղ', '资生堂套装', '组', '赫莲娜', '乔治·阿玛尼·西', '资生堂', '纪梵希不可抗拒', '永远补粉', '精华', '舒植村秀', '舒植村秀', '赫莲娜', '希思黎', '迪奥眼霜', '百优眼霜', '美宝莲适合我', '舒植村秀', '阿玛尼水上乐园', '卸妆膏', '娇韵诗', '资生堂', '雅诗兰黛', '阿玛尼', '娇韵诗双萃', '伊夫 圣罗兰', '娇兰', '阿玛尼', '调料', '倩碧', '兰蔻', '珠宝', '倩碧', '睡眠面膜', '雅诗兰黛高级夜间修护', '布鲁克', 'برجوا اير مات', '胶原蛋白', '娇韵诗', '纪梵希', '帕尔马', '阿玛尼寺', '眼部精华', '玛莎', '美容盒', '兰蔻', '刷子清洁剂', '海伦娜', '雅诗兰黛', '巴黎欧莱雅', '阿玛尼', '永远弥补', '资生堂', '香奈儿眼霜', '资生堂', '克莱尔·德·皮奥·贝蒂', '植村秀', '音量福特', '车身修理厂的油', '巴兹·伊卡雷', '娇韵诗双萃乳液', '倩碧', '罗伊·奥威', '纪梵希', '欧舒丹', '粉扑', '普通酸性痤疮皮血清', '迪奥精华', '兰蔻绝对', '阿玛尼', '勒韦独奏埃', '雅诗兰黛', '乔治阿玛尼', '资生堂', '定妆', '阿玛尼', '假指甲', '汤姆·福特·曼达里诺·迪阿玛·菲', '娇兰', '永远化

In [12]:
print(similarToCategory[0:20])

['zielenski rosen', 'подарочный набор', 'патчи', 'щипцы', 'тени', 'malin goeiz', 'подарочный набор', 'крем для лица', 'mainlypro', 'маска для лица', 'маска для лица', 'крем для лица', 'маска для лица', 'маска для лица', 'лак для ногтей', 'лак для ногтей', 'тени', 'сыворотка', 'блеск для губ', 'подарочный набор']


In [17]:
import pickle

with open('pickle_errorful.obj', 'wb') as pickle_errorful:
    pickle.dump(errorful, pickle_errorful)
with open('pickle_containingNumbers.obj', 'wb') as pickle_containingNumbers:
    pickle.dump(containingNumbers, pickle_containingNumbers)
with open('pickle_containingBrands.obj', 'wb') as pickle_containingBrands:
    pickle.dump(containingBrands, pickle_containingBrands)
with open('pickle_similarToCategory.obj', 'wb') as pickle_similarToCategory:
    pickle.dump(similarToCategory, pickle_similarToCategory)